Once you have trained a model and saved the output at "models/climate-classifier" you can use this code to run inference

In [10]:
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from scipy.special import softmax

In [14]:
model_dir = "models/climate-classifier"

# Get a list of all folders in model_dir that start with "checkpoint"
checkpoint_folders = [folder for folder in os.listdir(model_dir) if folder.startswith("checkpoint")]

# Sort the checkpoint folders by modification time in descending order
checkpoint_folders.sort(key=lambda folder: os.path.getmtime(os.path.join(model_dir, folder)), reverse=True)

# Get the most recent checkpoint folder
if checkpoint_folders:
    most_recent_checkpoint = os.path.join(model_dir, checkpoint_folders[0])
else:
    print("No checkpoint folders found in", model_dir)

tokenizer = AutoTokenizer.from_pretrained(most_recent_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(most_recent_checkpoint)

def inference(tokenizer, model, text):
    inputs = tokenizer(text, return_tensors="pt")

    with torch.no_grad():
        logits = model(**inputs).logits

    predicted_class_id = logits.argmax().item()
    class_name = model.config.id2label[predicted_class_id]
    predicted_confidences = softmax(logits[0], axis=0)
    class_confidence = predicted_confidences[predicted_class_id]
    print("Output: {}; Confidence: {}%".format(
        class_name,
        round(class_confidence * 100, 2)
      )
    )

In [15]:
text = "Fiduciary Risk Assessment of the Adaptation Fund"
inference(tokenizer, model, text)

Output: Related to climate; Confidence: 98.42%
